# 코드 정리


## 공통 함수 loading
- 여러가지 인사이트 및 metric 값을 계산하기 위한 함수들을 정의하여 사용하였습니다.  
  혹시 필요하시다면 참고하셔서 사용하셔도 좋을 듯 합니다. 최종 제출 코드에서는 RowVar 함수만 사용되었습니다.

  - outlier_IQRTr : 수치형 자료에서 IQR 값을 계산하기 위한 함수.
  - mkAUCValue    : testData에 대한 최종 AUC 값 평가에 사용한 함수.
  - getPackages   : package에 dependency가 있는 package를 로딩하기 위한 함수
  - RowVar        : 수치형 자료의 분산 값을 계산하기 위한 함수.

In [ ]:
outlier_IQRTr <- function(x){
  if(is.numeric(x)){
    x.stats <- boxplot.stats(x)$out
    x[x %in% x.stats] <- NA
    as.numeric(x)
  }else{
    x
  }
}

mkAUCValue <- function(YHat, Y){
  # levels(Y)    <- c(0, 1)
  # levels(YHat) <- c(0, 1)
  pred <- prediction(YHat, Y)
  # perf <- performance(pred, measure = "tpr", x.measure = "fpr")
  auc <- performance(pred, measure = "auc")
  auc@y.values[[1]]
}


getPackages <- function(packs){
  packages <- unlist(
    tools::package_dependencies(packs, available.packages(),
                                which=c("Depends", "Imports"), recursive=TRUE)
  )
  packages <- union(packs, packages)
  return(packages)
}


# row-wise variance
RowVar <- function(x, ...) {
  rowSums((x - rowMeans(x, ...))^2, ...)/(dim(x)[2] - 1)
}



## 라이브러리 & 데이터 로딩
- 데이터 로딩시 일반적으로 사용하는 read.csv 등 함수를 사용할 수 있지만,  
  빠른 데이터 로딩을 위하여 data.table library 내에 있는 fread 함수를 사용하였습니다

In [ ]:
library(DMwR);library(dplyr);library(data.table);library(caret);library(catboost);
library(Matrix);library(ROCR);library(CatEncoders);library(foreach);library(matrixStats)
##################
## Data Loading ##
##################
sample_submission <- data.table::fread(
  "../input/dacon-8th/sample_submission.csv",
  stringsAsFactors = F,
  data.table       = F
)

train <- data.table::fread(
  "../input/dacon-8th/train.csv",
  stringsAsFactors = F,
  data.table = F,
  na.strings = c("NA", "NaN", "NULL", "\\N"))

test  <- data.table::fread(
  "../input/dacon-8th/test_x.csv",
  stringsAsFactors = F,
  data.table = F,
  na.strings = c("NA", "NaN", "NULL", "\\N"))

## Feature Engineering 

### 1. 이상치 및 결측치 처리
- 분석 초반에는 Q_E와 familySize 변수들에 대해서 이상치 처리를 진행하였으나, 최종 평가에 있어 성능이 낮아져 최종 제출시 이상치 처리를 제외하였습니다.   
- 분석 초반에는 LightGBM + CatBoost 앙상블 조합으로 성능을 개선하려고 생각하였기에 decision Tree 기반 모형인 두 모형에 대해서 이상치는 크게 영향을 미치지
  않지 않을까 생각하여 제거하지 않았고, 그 이후에 특별한 조치는 하지 않았습니다  
  (개인적으로 시간이 있었다면 이 부분에서 좀 더 개선할 수 있지 않았나 생각합니다)

### 2. 파생변수 생성 및 변경
- 저는 개인적으로 파생변수 생성에 많은 신경을 썼습니다. 지금까지 개최되었던 많은 대회들의 상위 커널들을 살펴보면, 좋은 파생변수를 생성하여 모델의 성능을 좋게하는 경우가 굉장히 많다고 생각하였습니다.

- 우선 파생변수 생성 시 jhl님의 토론 공유 자료를 참고 하였습니다. 개인적으로 정말 신선한 접근 방식이라 많이 배울 수 있었습니다. 감사합니다.
  https://dacon.io/competitions/official/235647/codeshare/1711?page=1&dtype=recent&ptype=pub


- 파생변수 추가 여부를 판단하는 프로세스는 다음과 같습니다

  - 수치형 파생변수 같은 경우, 대선투표 결과 기준으로 T-Test 분석을 수행해 p-value 값이 0.05보다 작은지 확인(이산형은 카이제곱검정을 수행하려 했음)
  - 밀도 함수 그래프를 통하여 대선투표 결과 기준으로 차이가 나는지 시각적으로 확인(이산형은 막대 그래프)
  - 모델 생성 후 variable importance에 상위에 rank하는지 우선 확인. 모델에 거의 영향을 미치지 않는 경우는 제외.
  - 파생변수 추가 전 후로 하여 validationSet 및 testData(Public Score)의 AUC 값이 좋아진 경우, 해당 파생변수 추가 최종 결정.
  
- 판단에 있어 혼란스러웠던 점은 파생변수 추가 시 10-fold cross-validation 기반으로 AUC 값이 좋아져 추가했지만, 제출시에 Public Score 값이 낮아지는 경우가 꽤 있었습니다. 따라서 후반에는 Public Score 기준으로 파생변수 추가 여부를 최종 결정하였습니다.

- <b/> 최종 추가 파생변수 : QAvar, machiaScore, wf_mean, wr_mean, voca_mean, tp_positive, tp_negative, tp_var, tp_mean </b>

### 파생변수 List

- QAVar : Q_A 변수들의 분산 값을 계산.
- machiaScore : jhl님께서 공유해주신 마키아벨리니즘 테스트 스코어를 기반으로 '-' 에 해당하는 Q_A의 값을 reverse score 시킨 후(6-value), 평균 값 계산.
- wf_mean : wf_(01~03) 변수의 평균값 계산
- wr_mean : wr_(01~13) 변수의 평균값 계산
- voca_mean : 교육 수준에 따라 대선 투표 여부에 차이가 있다는 것을 기반으로 허구 단어는 모른다고 설문하고, 실제 존재하는 단어는 확실히 안다고 설문한다면 정직성 등을 감안해 어느정도 판단 지표로서 활용 가능하다고 생각하였습니다. (wf_(01~03) - wr(01-13)) / 16  
- tp_positive, tp_negative : 데이터 변수 설명에서 확인 결과, tp_(홀수번호)는 활발하고 에너지가 많은 성향을 가지는 부분이며  tp_(짝수번호)는 주로 조용하고 에너지가 덜한 성향을 가지는 부분임을 기반으로 홀수번호와 짝수번호들의 평균 값을 파생변수로 추가하였습니다.
- tp_var : tp_(01~10) 변수들의 분산 값 계산
- tp_mean : tp_(짝수번호)는 reverse score(7-value) 하여 전체 tp_(01~10)의 평균값 계산
- QE_median : Q_E의 중앙값 계산
- QE_min : Q_E의 최소값 계산

In [ ]:
# ###########################
# ## 파생변수 생성 및 변경 ##
# ###########################
#- 1. reverse 
#- QaA, QdA, QeA, QfA, QgA, QiA, QkA, QnA, QqA, QrA --> reverse 
revVar  <- c("QaA", "QdA", "QeA", "QfA", "QgA", "QiA", "QkA", "QnA", "QqA", "QrA")
train[revVar] <- train %>% select(revVar) %>% mutate_all(list(~6 - .))
test[revVar]  <- test %>% select(revVar) %>% mutate_all(list(~6 - .))

# 2. QAvar
QAvar <- train %>% select(matches("Q.A")) %>%  colnames
train$QA_var <- train %>% dplyr::select(c(QAvar)) %>% transmute(test = round(RowVar(across(where(is.numeric))), 4)) %>%  unlist %>% as.numeric
test$QA_var  <-  test %>% dplyr::select(c(QAvar)) %>% transmute(test = round(RowVar(across(where(is.numeric))), 4)) %>%  unlist %>% as.numeric

#- 2. machia score = 전체 점수의 평균 값 계산
machiaVar             <- train %>% select(matches("Q.A")) %>%  colnames
train$machiaScore     <- train %>% select(machiaVar) %>% transmute(machiaScore = rowMeans(across(where(is.numeric)))) %>% unlist %>% as.numeric
test$machiaScore      <- test  %>% select(machiaVar) %>% transmute(machiaScore = rowMeans(across(where(is.numeric)))) %>% unlist %>% as.numeric

#- 3 wf_mean, wr_mean, voca_mean(실제 단어를 아는 경우(wr)  - 허구인 단어를 아는 경우(wf) / 13)
wfVar <- train %>% select(matches("wf.")) %>%  colnames
wrVar <- train %>% select(matches("wr.")) %>%  colnames

#- 3.1 wf_mean
train$wf_mean <- train %>% select(wfVar) %>% transmute(wf_mean = round(rowMeans(across(where(is.numeric))), 8)) %>% unlist %>% as.numeric
test$wf_mean  <- test %>% select(wfVar)  %>% transmute(wf_mean = round(rowMeans(across(where(is.numeric))), 8)) %>% unlist %>% as.numeric

#- 3.2 wr_mean
train$wr_mean <- train %>% select(wrVar) %>% transmute(wr_mean = round(rowMeans(across(where(is.numeric))), 8)) %>% unlist %>% as.numeric
test$wr_mean  <- test %>% select(wrVar)  %>% transmute(wr_mean = round(rowMeans(across(where(is.numeric))), 8)) %>% unlist %>% as.numeric

#- 3.3 voca_mean
train$voca_mean <- train %>% transmute(voca_mean = round((
    wr_01 + wr_02 + wr_03 + wr_04 + wr_05 + wr_06 + wr_07 + wr_08 + wr_09 + wr_10 + wr_11 + wr_12 + wr_13 - wf_01 - wf_02 - wf_03 / 16), 8)) %>% unlist %>% as.numeric
test$voca_mean <- test %>% transmute(voca_mean = round((wr_01 + wr_02 + wr_03 + wr_04 + wr_05 + wr_06 + wr_07 + wr_08 + wr_09 + wr_10 + wr_11 + wr_12 + wr_13 - wf_01 - wf_02 - wf_03 / 16), 8)) %>% unlist %>% as.numeric

#- tp variable
tpPs <- c("tp01", "tp03", "tp05", "tp07", "tp09")
tpNg <- c("tp02", "tp04", "tp06", "tp08", "tp10")

#- 3.4 tp_positive
train$tp_positive  <- train %>% select(all_of(tpPs)) %>% transmute(tp_positive = round(rowMeans(across(where(is.numeric))), 8)) %>%  unlist %>% as.numeric 
test$tp_positive   <- test  %>% dplyr::select(all_of(tpPs)) %>% transmute(tp_positive = round(rowMeans(across(where(is.numeric))), 8)) %>%  unlist %>% as.numeric 

#- 3.5 tp_negative 
train$tp_negative  <- train %>% dplyr::select(all_of(tpNg)) %>% transmute(tp_negative = round(rowMeans(across(where(is.numeric))), 8)) %>%  unlist %>% as.numeric 
test$tp_negative   <- test  %>% dplyr::select(all_of(tpNg)) %>% transmute(tp_negative = round(rowMeans(across(where(is.numeric))), 8)) %>%  unlist %>% as.numeric 

#- 3.6 tp_variance
train$tp_var       <- train %>% dplyr::select(c(tpPs, tpNg)) %>% transmute(test = round(RowVar(across(where(is.numeric))), 4)) %>%  unlist %>% as.numeric 
test$tp_var        <- test %>% dplyr::select(c(tpPs, tpNg)) %>% transmute(test = round(RowVar(across(where(is.numeric))), 4)) %>%  unlist %>% as.numeric 

#- 3.7 tp_mean
train$tp_mean <- train %>% transmute(tp_mean = round(((tp01 + tp03 + tp05 + tp07 + tp09 + (7 - tp02) + (7 - tp04) + (7 - tp06) + (7 - tp08) + (7 - tp10)) / 10), 8)) %>%  unlist %>% as.numeric
test$tp_mean  <- test %>% transmute(tp_mean = round(((tp01 + tp03 + tp05 + tp07 + tp09 + (7 - tp02) + (7 - tp04) + (7 - tp06) + (7 - tp08) + (7 - tp10)) / 10), 8)) %>%  unlist %>% as.numeric

#- 4. QE derived Var
#- 4.1 QE_median
# QE_var          <- train %>% select(matches("Q.E")) %>%  colnames
# train$QE_median <- apply(train[, QE_var], 1,  median)
# test$QE_median  <- apply(test[, QE_var], 1,   median)

# #- 4.2 QE_min
# train$QE_min <- apply(train[, QE_var], 1,  min)
# test$QE_min  <- apply(test[, QE_var], 1,   min)

### 3. 변수타입설정 & 변수 선택
- 범주형 타입으로 변환이 필요한 변수들을 변환하였습니다.  
- 구분 짓기는 어려웠지만, 범주형을 다시 명목형과 순서형으로 변환하였습니다  
  구분한 이유는 CatBoost 모형 생성시 내부적으로 각각의 타입에 따라 다른 방식으로 encoding을 할 수도 있겠다고 판단하여 구분지었습니다.(큰 의미는 없지 않나 생각됩니다)
- Index 변수는 제거하였습니다.
- 한가지 특징적인 부분 중 하나는 파생변수로 생성하였던 QA_var 변수를 순서형 변수에 포함하였습니다.  
  성능 판단시에 더 좋은 성능을 나타내어 순서형 변수로 추가하였습니다.  

  
### 추가 특이사항
- 초기에는 one-hot encoding이나 label encoding을 한 후에 모델링을 진행했으나,  
  encoding을 하지 않고 모델링 시 범주형 변수를 지정하여 모델링 하는 것이 더 성능이 좋았습니다  
  (CatBoost 모델 특성상 내부적으로 범주형 변수인 경우 encoding을 수행하기 때문입니다)


In [ ]:
##############################
## 변수타입설정 & 변수 선택 ##
##############################
#- 수치형 변수 

#- 범주형(명목형) 변환
factor_var <- c("engnat",
                "age_group",
                "education",
                "gender",
                "hand",
                "married",
                "race",
                "religion",
                "urban",
                "voted")

yIdx <- which(factor_var %in% c('voted'))

train[factor_var]        <- train %>% dplyr::select(all_of(factor_var))        %>% mutate_all(as.factor)
test[factor_var[-yIdx]]  <-  test %>% dplyr::select(all_of(factor_var[-yIdx])) %>% mutate_all(as.factor)

#- 범주형(순서형) 변환
ordered_var1 <- colnames(train)[grep("Q.A", colnames(train))]
ordered_var2 <- colnames(train)[grep("tp|wr|wf.", colnames(train))]
ordered_var2 <- c(ordered_var2, "QA_var")

train[c(ordered_var1, ordered_var2)]   <- train %>% dplyr::select(all_of(ordered_var1), all_of(ordered_var2)) %>% mutate_all(as.ordered)
test[c(ordered_var1, ordered_var2) ]   <- test %>% dplyr::select(all_of(ordered_var1), all_of(ordered_var2)) %>% mutate_all(as.ordered)

#-  변수 제거
remv_var <- c("index")
train    <- train %>%  dplyr::select(-all_of(remv_var))
test     <- test  %>%  dplyr::select(-all_of(remv_var))

## Modeling

### 모델링 프로세스
- 모델링 프로세스는 다음과 같습니다
  - hyper parameter 조합별 모델 생성
  - variable importance 상위 70개 변수 선택
  - 70개 변수로 재 모델링
  - 성능이 가장 잘나온 상위 3개 Catboost 모형 선택
  - 3개 모형 앙상블
  
- 최종 소스에는 담겨있지 않지만, learningRate, maxDepth, l2_leaf_reg 3가지  
  hyper parameter에 대해서 조합별로 AUC가 가장 높은 3개의 CatBoost 모형을 선택하였습니다.
  
- 상위 70개의 변수를 선택하였을 때 모델의 성능이 더 좋아졌습니다

In [ ]:
############
## 모델링 ##
############
trainData <- train
testData  <- test

#################
## 2. CatBoost ##
#################
##- 기본 catBoost function을 이용한 모델 생성

# voted  1 --> 0, 2 --> 1로 변경 
trainData_cat <- trainData
testData_cat  <- testData
YIdx       <- which(colnames(trainData_cat) %in% c('voted'))
features   <- trainData_cat[-YIdx]
labels     <- ifelse(trainData_cat[,YIdx] == 1, 0, 1)
train_pool <- catboost.load_pool(data  = features, 
                                 label = labels)

- 모델 3개에 대해서 각각 모델링 -> 상위 70개 변수 선택 -> 재 모델링을 진행해야 하지만, 모델링 수행 시간이 너무 오래 걸려  
  1개의 모델만 가지고 variable importance 기준 상위 70개 변수를 선택하고 해당 변수로 3개 모델을 모델링 하였습니다. 

In [ ]:
grid <- data.frame(
  learningRate = c(0.01),
  maxDepth     = c(4), 
  l2_leaf_reg  = c(3.5)
)

- 상위 70개 변수 선택

In [ ]:
g <- grid[1, ]

  depth             <- g$maxDepth
  learning_rate     <- g$learningRate
  l2_leaf_reg       <- g$l2_leaf_reg
  
  
  # 2. catboost.train 함수를 이용하여 train
  set.seed(1)
  model <- catboost.train(
    train_pool,                                  #- 학습에 사용하고자 하는 train_pool  
    NULL,                                        #- 
    params = list(loss_function = 'Logloss',     #- loss function 지정(여기서는 분류모형이므로 Logloss)
                  random_seed   = 123,           #- seed number
                  custom_loss   = "AUC",         #- 모델링 할 때 추가로 추출할 값들 (train_dir로 지정한 곳으로 해당 결과를 파일로 내보내준다)
                #  train_dir     = "./model/CatBoost_R_output", #- 모델링 한 결과를 저장할 directory
                  iterations    = 1000,                         #- 학습 iteration 수
                  border_count  = 32,
                  depth         = depth,
                  learning_rate = learning_rate,
                  l2_leaf_reg   = l2_leaf_reg,
                  metric_period = 10)            
  )         
  
  
  # catboost importance 
  catboost_imp           <- data.frame(model$feature_importances)
  catboost_imp$variables <- rownames(model$feature_importances)
  colnames(catboost_imp) <- c("importance", 'variables')
  catboost_imp           <- catboost_imp %>% arrange(-importance)
  
  finalVar <- catboost_imp$variables[1:70] # 70개 변수 선택
  
  trainData <- train[c(finalVar, "voted")]
  testData  <- test[c(finalVar)]


- CatBoost 3개 모형 앙상블
- 좀 더 정밀한 예측을 위하여, 최종 값을 확률로 편성하였습니다.

In [ ]:
grid <- data.frame(
  learningRate = c(0.03, 0.03, 0.02),
  maxDepth     = c(7, 5, 6), 
  l2_leaf_reg  = c(5, 7.5, 3.5)
)

In [ ]:
testResult_Catboost <- foreach(i = 1:nrow(grid), .combine = function(a,b){ cbind(a, b)}) %do% {

  g <- grid[i, ]
  depth             <- g$maxDepth
  learning_rate     <- g$learningRate
  l2_leaf_reg       <- g$l2_leaf_reg
  
  
  # voted  1 --> 0, 2 --> 1로 변경 
  trainData_cat <- trainData
  testData_cat  <- testData
  YIdx       <- which(colnames(trainData_cat) %in% c('voted'))
  features   <- trainData_cat[-YIdx]
  labels     <- ifelse(trainData_cat[,YIdx] == 1, 0, 1)
  train_pool <- catboost.load_pool(data  = features, 
                                   label = labels)
    
    
  # 2. catboost.train 함수를 이용하여 train
  set.seed(1)
  model <- catboost.train(
    train_pool,                                  #- 학습에 사용하고자 하는 train_pool  
    NULL,                                        #- 
    params = list(loss_function = 'Logloss',     #- loss function 지정(여기서는 분류모형이므로 Logloss)
                  random_seed   = 123,           #- seed number
                  custom_loss   = "AUC",         #- 모델링 할 때 추가로 추출할 값들 (train_dir로 지정한 곳으로 해당 결과를 파일로 내보내준다)
                 # train_dir     = "./model/CatBoost_R_output", #- 모델링 한 결과를 저장할 directory
                  iterations    = 1000,                         #- 학습 iteration 수
                  border_count  = 32,
                  depth         = depth,
                  learning_rate = learning_rate,
                  l2_leaf_reg   = l2_leaf_reg,
                  metric_period = 10)            
  )         
    
  real_pool  <- catboost.load_pool(testData_cat)
  YHat_cat   <- catboost.predict(
    model, 
    real_pool,
    prediction_type = c('Probability'))  # Probability, Class
    
  gridCom       <- paste0(depth, "_", learning_rate, "_", l2_leaf_reg)
  tmp           <- data.frame(YHat_cat)  
  colnames(tmp) <- gridCom
  tmp
}

In [ ]:
YHat_cat <- testResult_Catboost %>% transmute(finalScore = rowMeans(across(where(is.numeric)))) %>% unlist %>% as.numeric

In [ ]:
save(testResult_Catboost, file = "testResult_Catboost.RData")